[View in Colaboratory](https://colab.research.google.com/github/youqad/Neurorobotics_Project/blob/master/Inferring_Space_from_Sensorimotor_Dependencies.ipynb)

# Inferring Space from Sensorimotor Dependencies


## The Algorithm

1. Proprioceptive input is separated from exteroceptive input by noting that proprioceptive input remains silent when no motor commands are given, whereas exteroceptive input changes because of environmental change.

2. Estimation of the number of parameters needed to describe the variation in the exteroceptive inputs when only the environment changes. The algorithm issues no motor commands and calculates the covariance matrix of the observed environment-induced variations in sensory inputs. The dimension estimation is done by considering the eigenvalues of this covariance matrix. The eigenvalues $λ_i$ should fall into two classes: a class with values all equal to zero and a class with nonzero values. The two classes are separated by a clustering method (e.g. PCA). The number of nonzero eigenvalues is taken as the number of dimensions.

3. Estimation of the number of parameters needed to describe the variation in the exteroceptive inputs when only the body moved. The environment is kept fixed, and the algorithm gives random motor commands. The covariance matrix of the resulting changes is observed and the dimension is estimated from the number of nonzero eigenvalues in the same way as before.

4. Estimation of the number of parameters needed to describe the changes in exteroceptive inputs when both the body and the environment change. The environment is changed at random, and the organism gives random motor commands. The number of nonzero eigenvalues of the covariance matrix is obtained as before.

## Simulation

### Notations

Notation|Meaning
-|-
$$Q ≝ (Q_1, \ldots, Q_{3q})$$|positions of the joints
$$P ≝ (P_1, \ldots, P_{3p})$$|positions of the eyes
$$a^θ_i, a^φ_i, a^ψ_i$$|Euler angles for the orientation of eye i
$$Rot(a^θ_i, a^φ_i, a^ψ_i)$$|rotation matrix for eye i
$$C_{i,k}$$|relative position of photosensor $k$ within eye $i$
$$d ≝ (d_1, \ldots,d_p)$$|apertures of diaphragms
$$L ≝ (L_1,\ldots,L_{3r})$$|positions of the lights
$$θ ≝ (θ_1, \ldots, θ_r)$$|luminances of the lights
$$S^e_{i,k}$$|sensory input from exteroceptive sensor $k$ of eye $i$
$$S^p_i$$|sensory input from proprioceptive sensor $i$
$$M, E$$|motor command and environmental control vector


### Computing the sensory inputs


$$
\begin{align*}
(Q,P,a) &≝ σ(W_1 · σ(W_2 · M − μ_2)−μ_1)\\
L &≝ σ(V_1 ·σ(V_2 · E − ν_2) − ν_1)\\
∀1≤ k ≤ p', 1≤i≤p, \quad S^e_{i,k} &≝ d_i \sum\limits_{ j } \frac{θ_j}{\Vert P_i + Rot(a_i^θ, a_i^φ, a_i^ψ) \cdot C_{i,k} - L_j \Vert^2}\\
(S^p_i)_{1≤ i ≤ q'q} &≝ σ(U_1 · σ(U_2 · Q − τ_2) − τ_1)
\end{align*}
$$

where

- $W_1, W_2, V_1, V_2, U_1, U_2$ are matrices with coefficients drawn randomly from a uniform distribution between $−1$ and $1$
- the vectors $μ_1, μ_2, ν_1, ν_2, τ_1, τ_2$ too
- $σ$ is an arbitrary nonlinearity (e.g. the hyperbolic tangent function)
- the $C_{i,k}$ are drawn from a centered normal distribution whose variance (which can be understood as the size of the retina) is so that the sensory changes resulting from a rotation of the eye are of the same order of magnitude as the ones resulting from a translation of the eye
- $θ$ and $d$ are constants drawn at random in the interval $[0.5, 1]$


In [2]:
# /!\ We're using Python 3

import plotly
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go

import matplotlib.pyplot as plt
import scipy.io as sio
from scipy import stats
from scipy import special
from scipy.spatial import distance

from sklearn import preprocessing

%matplotlib inline

plotly.offline.init_notebook_mode(connected=True)
from IPython.core.display import display, HTML, Markdown
# The polling here is to ensure that plotly.js has already been loaded before
# setting display alignment in order to avoid a race condition.
display(HTML(
    '<script>'
        'var waitForPlotly = setInterval( function() {'
            'if( typeof(window.Plotly) !== "undefined" ){'
                'MathJax.Hub.Config({ SVG: { font: "STIX-Web" }, displayAlign: "center" });'
                'MathJax.Hub.Queue(["setRenderer", MathJax.Hub, "SVG"]);'
                'clearInterval(waitForPlotly);'
            '}}, 250 );'
    '</script>'
))

# Colorscales
def colorscale_list(cmap, number_colors, return_rgb_only=False):
    cm = plt.get_cmap(cmap)
    colors = [np.array(cm(i/number_colors)) for i in range(1, number_colors+1)]
    rgb_colors_plotly = []
    rgb_colors_only = []
    for i, c in enumerate(colors):
        col = 'rgb{}'.format(tuple(255*c[:-1]))
        rgb_colors_only.append(col)
        rgb_colors_plotly.append([i/number_colors, col])
        rgb_colors_plotly.append([(i+1)/number_colors, col])
    return rgb_colors_only if return_rgb_only else rgb_colors_plotly

from scipy.io import loadmat, whosmat
from numpy.random import randint

def formatted(f): 
    return format(f, '.2f').rstrip('0').rstrip('.')

## Organism 1

**Parameter**|**Value**
-|-
Dimension of motor commands|$40$
Dimension of environmental control vector|$40$
Dimension of proprioceptive inputs|$16 \quad (= 4×4)$
Dimension of exteroceptive inputs|$40 \quad (= 2 × 20)$
Number of eyes|$2$
Number of joints|$4$
Diaphragms|None
Number of lights|$3$
Light luminance|Fixed


1. The arm has

    - $4$ joints

        - each of which has $4$ **proprioceptive** sensors (whose output depended on the position of the joint)
    - $2$ eyes (for each of them: $3$ spatial and $3$ orientation coordinates)

        - on which there are $20$ omnidirectionally sensitive photosensors (**exteroceptive**)
2. the motor command is $40$-dimensional  

3. the environment consists of:

    - $3$ lights ($3$ spatial coordinates and $3$ luminance values for each of them)

In [0]:
#@title Parameters

M_size = 40 #@param {type:"slider", min:10, max:100, step:5}
E_size = 40 #@param {type:"slider", min:10, max:100, step:5}

# Number of Joints / q
nb_joints = 4 #@param {type:"slider", min:3, max:5, step:1}

# Number of eyes / p
nb_eyes = 2 #@param {type:"slider", min:2, max:5, step:1}

# Number of lights / r
nb_lights = 3 #@param {type:"slider", min:2, max:5, step:1}

# Number of exteroceptive photosensors / p'
extero = 20 #@param {type:"slider", min:15, max:25, step:1}

# Number of proprioceptive sensors / q'
proprio = 4 #@param {type:"slider", min:3, max:5, step:1}


# Sensory inputs were generated from...
nb_generating_motor_commands = 50 #@param {type:"slider", min:10, max:200, step:10}
nb_generating_env_positions = 50 #@param {type:"slider", min:10, max:200, step:10}

# Neighborhood size of the linear approximation:
# Motor commands/Environmental positions drawn from normal distribution
# with mean zero and standard deviation... 
neighborhood_size = 1e-8
# (Coordinates differing from 0 by more than the std deviation are set equal to 0)

sigma = np.tanh

In [0]:
# Rotation matrix computation

def Rot(euler_angles):
  R = np.empty((3,3))
  c = np.cos(euler_angles)
  s = np.sin(euler_angles)

  # cf. https://en.wikipedia.org/wiki/Euler_angles#Rotation_matrix 
  R[0,0] = c[0]*c[2] - c[1]*s[0]*s[2]
  R[0,1] = -c[0]*s[2] - c[1]*c[2]*s[0]
  R[0,2] = s[0]*s[1]

  R[1,0] = s[0]*c[2] + c[0]*c[1]*s[2]
  R[1,1] = c[0]*c[1]*c[2] - s[0]*s[2]
  R[1,2] = -c[0]*s[1]

  R[2,0] = s[1]*s[2]
  R[2,1] = -c[2]*s[1]
  R[2,2] = c[1]

  return R


def PCA(data):
  cov_matrix = np.cov(preprocessing.scale(data.T))
  eig_val, _ = np.linalg.eigh(cov_matrix)
  
  #print(cov_matrix)
  #print([eig_val[i+1]/eig_val[i] for i in range(len(eig_val)-1)])
  
  max_ratio = np.argmax([eig_val[i+1]/eig_val[i] for i in range(len(eig_val)-1)])
  
  return len(eig_val[max_ratio+1:])


nb_generating_motor_commands = 50
nb_generating_env_positions = 50

In [0]:
class Organism1:
  """
  Organism 1:
  
    1. The arm has

      - `nb_joints` joints

          - each of which has `proprio` **proprioceptive** sensors 
          (whose outputs depend on the position of the joint)
          
      - `nb_eyes` eyes (for each of them: 3 spatial and 3 orientation coordinates)

          - on which there are `extero` omnidirectional **exteroceptive** photosensors
          
    2. the motor command is `M_size`-dimensional  

    3. the environment consists of:

        - `nb_lights` lights 
          (3 spatial coordinates and `nb_lights` luminance values for each of them)
        
    Other parameters:
    
    - Random seed: `seed`
    
    - Sensory inputs are generated from `nb_generating_motor_commands` motor commands
      and `nb_generating_env_positions` environment positions
      
    - Neighborhood size of the linear approximation: `neighborhood_size`
      i.e. Motor commands/Environmental positions drawn from normal distribution
      with mean zero and standard deviation `neighborhood_size`
      (Coordinates differing from 0 by more than the std deviation are set equal to 0)
      
    - `retina_size` size of the retina: variance of the normal distribution from
      which are drawn the C[i,k] (relative position of photosensor k within eye i)
      
  """
  def __init__(self, seed=1, retina_size=1., M_size=M_size, E_size=E_size,
               nb_joints=nb_joints, nb_eyes=nb_eyes, nb_lights=nb_lights, 
               extero=extero, proprio=proprio, 
               nb_generating_motor_commands=nb_generating_motor_commands,
               nb_generating_env_positions=nb_generating_env_positions,
               neighborhood_size=neighborhood_size, sigma=sigma):

    self.random = np.random.RandomState(seed)

    #------------------------------------------------------------
    # Random initializations

    dim_mu1 = 3*nb_joints+3*nb_eyes+3*nb_eyes
    W_1, mu_1 = 2*self.random.rand(dim_mu1, dim_mu1)-1, 2*self.random.rand(dim_mu1)-1
    W_2, mu_2 = 2*self.random.rand(dim_mu1, M_size)-1, 2*self.random.rand(dim_mu1)-1

    dim_nu1 = 3*nb_lights
    V_1, nu_1 = 2*self.random.rand(dim_nu1, dim_nu1)-1, 2*self.random.rand(dim_nu1)-1
    V_2, nu_2 = 2*self.random.rand(dim_nu1, E_size)-1, 2*self.random.rand(dim_nu1)-1

    dim_tau1 = proprio*nb_joints
    U_1, tau_1 = 2*self.random.rand(dim_tau1, dim_tau1)-1, 2*self.random.rand(dim_tau1)-1
    U_2, tau_2 = 2*self.random.rand(dim_tau1, 3*nb_joints)-1, 2*self.random.rand(dim_tau1)-1

    theta, d = .5*self.random.rand(nb_lights)+.5, .5*self.random.rand(nb_eyes)+.5

    C = self.random.multivariate_normal(np.zeros(3), retina_size*np.identity(3), (nb_eyes, extero))

    M_0, E_0 = 10*self.random.rand(M_size)-5, 10*self.random.rand(E_size)-5

    #------------------------------------------------------------
    # Setting attributes

    self.__dict__.update((key, value) 
                         for key, value in locals().items() if key != 'self')

    self.dim_mu1, self.W_1, self.mu_1, self.W_2, self.mu_2 = dim_mu1, W_1, mu_1, W_2, mu_2
    self.dim_nu1, self.V_1, self.nu_1, self.V_2, self.nu_2 = dim_nu1, V_1, nu_1, V_2, nu_2
    self.dim_tau1, self.U_1, self.tau_1, self.U_2, self.tau_2 = dim_tau1, U_1, tau_1, U_2, tau_2
    self.theta, self.d, self.C, self.M_0, self.E_0 = theta, d, C, M_0, E_0

    self.self_table = '''**Characteristics**|**Value**
    -|-
    Dimension of motor commands|{}
    Dimension of environmental control vector|{}
    Dimension of proprioceptive inputs|{}
    Dimension of exteroceptive inputs|{}
    Number of eyes|{}
    Number of joints|{}
    Diaphragms|None
    Number of lights|{}
    Light luminance|Fixed
    '''.format(M_size, E_size, proprio*nb_joints, extero*nb_eyes, nb_eyes,
              nb_joints, nb_lights)
    
    self.random_state = self.random.get_state()

  
  def get_sensory_inputs(self, M, E):
    """
    Compute sensory inputs for motor command M and environment position E
    """
    Q, P, a = [arr.reshape([-1, 3]) 
               for arr in np.split(
                   self.sigma(self.W_1.dot(self.sigma(self.W_2.dot(M)-self.mu_2))-self.mu_1),
                   [3*self.nb_joints, 3*self.nb_joints+3*self.nb_eyes])]
    
    L = self.sigma(self.V_1.dot(self.sigma(self.V_2.dot(E)-self.nu_2))-self.nu_1).reshape([-1, 3])
    Sp = self.sigma(self.U_1.dot(self.sigma(self.U_2.dot(Q.flatten())-self.tau_2))-self.tau_1)
    Se = np.array([self.d[i]*
                   sum(self.theta[j]/np.linalg.norm(P[i]+Rot(a[i]).dot(self.C[i,k])-L[j])**2
                       for j in range(self.nb_lights))
                   for i in range(self.nb_eyes)
                   for k in range(self.extero)])
    return np.concatenate((Sp, Se))

  
  def compute_proprioception(self):
    """
    Computes a mask indicating the sensory inputs the organism can reliably
    deem to be proprioceptive, since they remain silent when:
    - the motor command is fixed
    - the environment changes
    
    Useful to separate proprioceptive inputs from exteroceptive ones
    """
    self.random.set_state(self.random_state)
    
    # Separating proprioceptive input from exteroceptive input
    mask_proprio = np.array([
        self.get_sensory_inputs(self.M_0,
                                self.E_0+self.random.normal(0, self.neighborhood_size, self.E_size))
        for _ in range(self.nb_generating_env_positions)])
    self.mask_proprio = np.all(mask_proprio == mask_proprio[0, :], axis=0)
    
    self.random_state = self.random.get_state()
      
  def _neighborhood_lin_approx(self, size):
    self.random.set_state(self.random_state)
    
    rand_vect = self.random.normal(0, self.neighborhood_size, size)
    rand_vect[np.abs(rand_vect) > self.neighborhood_size] = 0
    
    self.random_state = self.random.get_state()
    
    return rand_vect

  
  def compute_variations(self):
    """
    Compute the variations in the exteroceptive inputs when:
    1. only the environment changes
    2. only the motor commands change
    3. both change
    """
    self.env_variations = np.array([
        self.get_sensory_inputs(self.M_0,
                                 self.E_0+self._neighborhood_lin_approx(self.E_size))[~self.mask_proprio]
        for _ in range(self.nb_generating_env_positions)])

    self.mot_variations = np.array([
        self.get_sensory_inputs(self.M_0+self._neighborhood_lin_approx(self.M_size),
                                 self.E_0)[~self.mask_proprio]
        for _ in range(self.nb_generating_motor_commands)])

    self.env_mot_variations = np.array([
        self.get_sensory_inputs(self.M_0+self._neighborhood_lin_approx(self.M_size),
                                 self.E_0+self._neighborhood_lin_approx(self.E_size))[~self.mask_proprio]
        for _ in range(self.nb_generating_motor_commands*self.nb_generating_env_positions)])
        
  def get_dimensions(self):
    """
    Compute the number of parameters needed to describe the exteroceptive variations when:
    1. only the environment changes
    2. only the body moves
    3. both the body and the environment change
    
    and then compute the estimated dimension of the group of compensated movements
    """
    self.compute_proprioception()
    self.compute_variations()
    
    # Now PCA!
    self.dim_env = PCA(self.env_variations)
    self.dim_extero = PCA(self.mot_variations)
    self.dim_env_extero = PCA(self.env_mot_variations)
    self.dim_rigid_group = self.dim_env+self.dim_extero-self.dim_env_extero

    self.dim_table = '''
    **Characteristics**|**Value**
    -|-
    '''

    end_table = '''Dimension for body (p)|{}
    Dimension for environment (e)|{}
    Dimension for both (b)|{}
    Dimension of group of compensated movements|{}
    '''.format(self.dim_extero, self.dim_env, self.dim_env_extero, self.dim_rigid_group)
    
    self.dim_table += end_table
    self.self_table += end_table

    return self.dim_rigid_group, self.dim_extero, self.dim_env, self.dim_env_extero
  
  def __str__(self):
        return self.self_table

In [25]:
seed = 3
sim = []

for var in np.arange(.1, 16.1, .1):
  val, bod, env, env_bod = Organism1(seed=seed, retina_size=var).get_dimensions()
  print("var = {:.1f} / dim of compensated mov. = {} ({} {} {})".format(var, val, bod, env, env_bod))
  sim.append(val)

sim = np.array(sim)
best_sim = np.argmax(sim)

var = 0.1 / dim of compensated mov. = 0 (5 1 6)
var = 0.2 / dim of compensated mov. = 0 (5 1 6)
var = 0.3 / dim of compensated mov. = 0 (5 1 6)
var = 0.4 / dim of compensated mov. = 0 (5 1 6)
var = 0.5 / dim of compensated mov. = 0 (5 1 6)
var = 0.6 / dim of compensated mov. = 0 (5 1 6)
var = 0.7 / dim of compensated mov. = 0 (5 1 6)
var = 0.8 / dim of compensated mov. = 0 (5 1 6)
var = 0.9 / dim of compensated mov. = 0 (5 1 6)
var = 1.0 / dim of compensated mov. = 0 (5 1 6)
var = 1.1 / dim of compensated mov. = 0 (5 1 6)
var = 1.2 / dim of compensated mov. = 0 (5 1 6)
var = 1.3 / dim of compensated mov. = 0 (5 1 6)
var = 1.4 / dim of compensated mov. = 0 (5 1 6)
var = 1.5 / dim of compensated mov. = 0 (5 1 6)
var = 1.6 / dim of compensated mov. = 0 (5 1 6)
var = 1.7 / dim of compensated mov. = 0 (5 1 6)
var = 1.8 / dim of compensated mov. = 0 (5 1 6)
var = 1.9 / dim of compensated mov. = 0 (5 1 6)
var = 2.0 / dim of compensated mov. = 0 (5 1 6)
var = 2.1 / dim of compensated mov. = 0 

var = 6.4 / dim of compensated mov. = 1 (6 1 6)
var = 6.5 / dim of compensated mov. = 1 (6 1 6)
var = 6.6 / dim of compensated mov. = 1 (6 1 6)
var = 6.7 / dim of compensated mov. = 1 (6 1 6)
var = 6.8 / dim of compensated mov. = 3 (6 3 6)
var = 6.9 / dim of compensated mov. = 3 (6 3 6)
var = 7.0 / dim of compensated mov. = 1 (6 1 6)
var = 7.1 / dim of compensated mov. = 3 (6 3 6)
var = 7.2 / dim of compensated mov. = 3 (6 3 6)
var = 7.3 / dim of compensated mov. = 3 (6 3 6)
var = 7.4 / dim of compensated mov. = 3 (6 3 6)
var = 7.5 / dim of compensated mov. = 3 (6 3 6)
var = 7.6 / dim of compensated mov. = 3 (6 3 6)
var = 7.7 / dim of compensated mov. = 1 (6 1 6)
var = 7.8 / dim of compensated mov. = 3 (6 3 6)
var = 7.9 / dim of compensated mov. = 3 (6 3 6)
var = 8.0 / dim of compensated mov. = 3 (6 3 6)
var = 8.1 / dim of compensated mov. = 3 (6 3 6)
var = 8.2 / dim of compensated mov. = 3 (6 3 6)
var = 8.3 / dim of compensated mov. = 3 (6 3 6)
var = 8.4 / dim of compensated mov. = 1 

var = 12.7 / dim of compensated mov. = 2 (7 1 6)
var = 12.8 / dim of compensated mov. = 2 (7 1 6)
var = 12.9 / dim of compensated mov. = 2 (7 1 6)
var = 13.0 / dim of compensated mov. = 2 (7 1 6)
var = 13.1 / dim of compensated mov. = 2 (7 1 6)
var = 13.2 / dim of compensated mov. = 2 (7 1 6)
var = 13.3 / dim of compensated mov. = 2 (7 1 6)
var = 13.4 / dim of compensated mov. = 2 (7 1 6)
var = 13.5 / dim of compensated mov. = 2 (7 1 6)
var = 13.6 / dim of compensated mov. = 2 (7 1 6)
var = 13.7 / dim of compensated mov. = 2 (7 1 6)
var = 13.8 / dim of compensated mov. = 2 (7 1 6)
var = 13.9 / dim of compensated mov. = 2 (7 1 6)
var = 14.0 / dim of compensated mov. = 2 (7 1 6)
var = 14.1 / dim of compensated mov. = 2 (7 1 6)
var = 14.2 / dim of compensated mov. = 2 (7 1 6)
var = 14.3 / dim of compensated mov. = 2 (7 1 6)
var = 14.4 / dim of compensated mov. = 2 (7 1 6)
var = 14.5 / dim of compensated mov. = 2 (7 1 6)
var = 14.6 / dim of compensated mov. = 2 (7 1 6)
var = 14.7 / dim of 

In [0]:
for var in best_sim:
  tab, val = simulation(seed, var)
  print("var = {} / dim of compensated mov. = {}".format(var, val))
  print(tab, "\n\n")

In [0]:
for seed in range(6):
  for var in range(1, 10):
    print(simulation(seed, var))

In [0]:
# Other Dimension Reduction Methods
## LDA is not applicable for our data
## Found codes for classical MDS so far -- 
## to be cordinated with our data

from __future__ import division
def mds(D):
    """                                                                                       
    Classical multidimensional scaling (MDS)                                                  
                                                                                               
    Parameters                                                                                
    ----------                                                                                
    D : (n, n) array                                                                          
        Symmetric distance matrix.                                                            
                                                                                               
    Returns                                                                                   
    -------                                                                                   
    Y : (n, p) array                                                                          
        Configuration matrix. Each column represents a dimension. Only the                    
        p dimensions corresponding to positive eigenvalues of B are returned.                 
        Note that each dimension is only determined up to an overall sign,                    
        corresponding to a reflection.                                                        
                                                                                               
    e : (n,) array                                                                            
        Eigenvalues of B.                                                                     
                                                                                               
    """
    # Number of points                                                                        
    n = len(D)
 
    # Centering matrix                                                                        
    H = np.eye(n) - np.ones((n, n))/n
 
    # YY^T                                                                                    
    B = -H.dot(D**2).dot(H)/2
 
    # Diagonalize                                                                             
    evals, evecs = np.linalg.eigh(B)
 
    # Sort by eigenvalue in descending order                                                  
    idx   = np.argsort(evals)[::-1]
    evals = evals[idx]
    evecs = evecs[:,idx]
 
    # Compute the coordinates using positive-eigenvalued components only                      
    w, = np.where(evals > 0)
    L  = np.diag(np.sqrt(evals[w]))
    V  = evecs[:,w]
    Y  = V.dot(L)
 
    return Y, evals


#Autoencoder: https://github.com/asdspal/dimRed/blob/master/autoencoder.ipynb